<img src="images/kiksmeisedwengougent.png" alt="Banner" width="1100"/>

<div>
    <font color=#690027 markdown="1">
        <h1>DE FUNDAMENTEN VAN EEN DIEP NEURAAL NETWERK VOOR BEELDHERKENNING</h1> 
    </font>
</div>

<div class="alert alert-box alert-success">
In deze notebook maak je stap voor stap kennis met de opbouw van een diep neuraal netwerk dat in staat is een onderscheid te maken tussen een afbeelding met een stoma en een afbeelding zonder stoma. <br>
    Het netwerk biedt een oplossing voor een <em>classificatieprobleem</em>: een afbeelding wordt geclassificeerd als 'Stoma' of 'Geen stoma'.
</div>

Bij de ontwikkeling van een neuraal netwerk moeten meerdere keuzes worden gemaakt en waarden van parameters worden vastgelegd.  Deze keuzes bepalen de architectuur van het netwerk en hoe de training van het netwerk verloopt. 

Normaal gezien duurt het even om een netwerk te trainen. De netwerken die in deze notebook gemaakt kunnen worden, zijn echter op voorhand getraind en de resultaten zijn opgeslagen in een databank. Op deze manier kan je dus onmiddellijk kijken naar de prestaties van het gekozen netwerk. 

<div class="alert alert-box alert-warning">
Wil je zelf een netwerk trainen, neem dan achteraf een kijkje in de notebook 'Van blad naar label' van het leerpad 'Gevorderd Deep Learning '. 
</div>

<div class="alert alert-block alert-warning"> 
Bekijk een concrete toepassing van classificatie in de notebook 'Stomata Zon Schaduw' van het leerpad 'ML Classificatie'.
</div>

### Nodige modules installeren en importeren

Voer onderstaande code-cellen uit om van de functies in deze notebook gebruik te kunnen maken.

In [ ]:
import sys
!{sys.executable} -m pip install pymongo

In [ ]:
import importlib.util
spec = importlib.util.spec_from_file_location(
name = "diep_neuraal_netwerk", 
location = ".scripts/diep_neuraal_netwerk.py"
)
diep_neuraal_netwerk = importlib.util.module_from_spec(spec)
spec.loader.exec_module(diep_neuraal_netwerk)

<div>
    <font color=#690027 markdown="1">
        <h2>1. De data</h2> 
    </font>
</div>

Een diep neuraal netwerk leert een invoer afbeelden op een uitvoer door **gelabelde data** te verwerken. Gelabelde data zijn data die bestaan uit invoer met telkens de daarbij behorende verwachte uitvoer van het model. <br><br>
Voor het stomataprobleem zijn de data microfoto's van delen van bladeren van verschillende soorten planten uit het tropisch regenwoud; bij elke microfoto hoort een label dat vermeldt of de foto al dan niet een stoma toont. De foto's zijn dus opgedeeld in **twee klassen** die de uitvoer van het netwerk voorstellen.<br>
Al deze microfoto's zijn kleurenfoto's met een formaat van **120 x 120 pixels**. Zoals te zien op Figuur 1, behoren de foto's met een stoma in het midden tot de klasse 'Stoma'; de foto's zonder een stoma, met een gedeeltelijke stoma of met een stoma die niet in het midden van de afbeelding staat, behoren tot de klasse 'Geen stoma'. Bij foto's met een stoma vult de stoma het grootste deel van de foto.

<img src="images/trainingdata.jpg"/>
<center>Figuur 1: Voorbeeld van de gelabelde data: de invoer en de daarbij behorende uitvoer.</center>

<b>Er is experimenteel bepaald dat er om een zo goed mogelijk resultaat te bekomen 6 keer meer foto's zonder een stoma nodig zijn dan foto's  met een stoma. </b>Dit komt omdat er meer variëteit zit in de afbeeldingen zonder stoma, waardoor er dus ook meer voorbeelden nodig zijn.

De beschikbare data worden verdeeld in 3 groepen:
<ul>
    <li><b>Trainingset</b>: Dit zijn de data die gebruikt worden om het model te trainen.<br>76 740 foto's met label 'Geen stoma' + 12 790 foto's met label 'Stoma' = 89 530 trainingsafbeeldingen</li>
    <li><b>Valideringsset</b>: Deze data worden gebruikt om na te gaan hoe goed het netwerk presteert op data die het nog niet gezien heeft. Op basis van deze data wordt het netwerk bijgeschaafd om betere resultaten te bekomen.<br>28 866 foto's met label 'Geen stoma' + 4 811 foto's met label 'Stoma'  = 33 677 valideringsafbeeldingen</li>
    <li><b>Testset</b>: Na het trainen met de trainingset en bijschaven van het netwerk met de valideringsset wordt het netwerk nog één keer geëvalueerd aan de hand van de testset. <br>55 182 foto's met label 'Geen stoma' + 9 197 foto's met label 'Stoma' = 64 379 test afbeeldingen</li>
</ul>

Het lijkt misschien overbodig om een testset te hebben om het netwerk nog een laatste keer te evalueren, omdat men a.d.h.v. de valideringsset al nagaat hoe goed het netwerk presteert. Toch is dit noodzakelijk. <br>De resultaten van de valideringsset worden immers gebruikt om het netwerk zo aan te passen dat het beter presteert op deze valideringsset. Op die manier zoekt men het beste netwerk voor een specifieke valideringsset. Hoe het netwerk uiteindelijk presteert op de valideringsset, is ongeschikt om in te schatten hoe het netwerk presteert op nieuwe data; de foto's van de valideringsset zijn immers geen nieuwe data meer. **Om de prestatie van het uiteindelijke netwerk te beoordelen, is de mate waarin het netwerk presteert op een testset die het netwerk nog nooit gezien heeft, aangewezen.**

<div>
    <font color=#690027 markdown="1">
        <h2>2. De netwerkarchitectuur</h2> 
    </font>
</div>

Een diep neuraal netwerk bestaat uit verschillende opeenvolgende lagen die de invoer, laag na laag, omzetten in de uitvoer. Hoe meer lagen het netwerk telt, hoe dieper het netwerk is.

Figuur 2 toont een voorbeeld van een diep neuraal netwerk. 
<img src="images/vbnetwerk.jpg"/>
<center>Figuur 2: De opbouw van een neuraal netwerk.</center>

Het netwerk is een feedforward netwerk. De invoer wordt erdoor gestuurd van links naar rechts. De eerste (blauwe) lagen zijn convolutionele lagen en de andere (paarse) lagen zijn *dense layers*.

**De volgende subparagrafen beschrijven elk een deel van het netwerk. Het cijfer 1 op de afbeelding verwijst dus naar paragraaf 2.1, het cijfer 2 naar paragraaf 2.2, enz.**

<div>
    <font color=#690027 markdown="1">
        <h3>2.1 Invoer</h3> 
    </font>
</div>

Zoals eerder vermeld, wordt in het netwerk een microfoto ingevoerd van een deel van een blad waarop wel of juist geen stoma te zien is, en heeft deze kleurenfoto een formaat van 120 x 120 pixels. De foto zal (digitaal) worden voorgesteld als een 3D-tensor met dimensie 3x120x120, zodat hij verwerkt kan worden door het netwerk.

<div class="alert alert-block alert-warning"> 
Hoe een computer naar foto's kijkt, lees je in de notebooks 'Matrices en afbeeldingen in grijswaarden' en en 'Tensoren en RGB'. Voor uitleg over tensoren kan je terecht in de notebooks 'Tensoren'. Deze STEM-notebooks vind je in het leerpad "Digitale beelden'.
</div>

<div>
    <font color=#690027 markdown="1">
        <h3>2.2 Convolutionele lagen</h3> 
    </font>
</div>

De convolutionele lagen dienen om relevante patronen te ontdekken in de ingevoerde afbeelding (voorgesteld door een tensor). 

Een convolutionele laag zal een tweede 3D-tensor, een (<b>filter</b>), met dimensie 3xaxb, linksboven op de afbeelding leggen en een wiskundige berekening, een convolutie, uitvoeren, die één getal teruggeeft. Hierna zal deze filter een bepaald aantal pixels opschuiven en wordt het volgende getal berekend. Het resultaat is een matrix die informatie bevat over waar het patroon van de filter voorkomt in de invoer. De persoon die het netwerk ontwerpt, kiest de waarde van a en van b en het aantal pixels waarover opgeschoven wordt.  

Het netwerk voor de classificatie van stomata gebruikt filters van 3 op 3, met diepte 3, en deze filters zullen steeds 1 positie per keer opschuiven.

De volgende afbeelding toont deze operatie op een afbeelding van 8 x 8 pixels met diepte 3 (RGB, dimensie 3x8x8) en met als filter een 3D-tensor met dimensie 3x3x3 die steeds één positie opschuift. De bekomen matrix heeft dimensie 6x6.

<img src="images/convoperation.jpg" width="500"/>
<center>Figuur 3: Convolutie.</center>

Een convolutionele laag zal meestal niet één maar meerdere filters gebruiken om verschillende patronen te herkennen. 
-  Elke filter die over de afbeelding glijdt, heeft een matrix als resultaat. Al deze matrices hebben dezelfde dimensie. 
-  Door deze matrices samen te stellen, ontstaat er een nieuwe 3D-tensor waarbij een van de getallen in de dimensie gelijk is aan het aantal filters. Deze tensor wordt een **feature map** genoemd. 
-  De elementen van deze filters worden aangepast tijdens het trainen van het model, met als doel het herkennen van relevante patronen. 

In Figuur 2 stelt het getal onder een convolutionele laag (bij de eerste is dat 32) het aantal filters voor. Voor filters met dimensie 3x3x3 zal de uitvoertensor van de eerste laag, dus de feature map van de eerste laag, een dimensie 32x118x118 hebben.

<img src="images/convlayer2.jpg"/>
<center>Figuur 4: Convolutionele laag.</center>

<div class="alert alert-block alert-warning"> 
Het verrassende effect van convoluties ondervind je in de notebook 'Convolutie' van het leerpad 'Deep learning basis'. Om te weten welke bewerking er wordt uitgevoerd, bekijk je de notebook 'Convolutie: de bewerking' van het leerpad 'Gevorderd Deep Learning'.
</div>

<div>
    <font color=#690027 markdown="1">
            <h3>2.3 Max pooling</h3> 
    </font>
</div>

De convolutionele lagen worden afgewisseld met max pooling-operaties. Deze operaties houden van een venster met zelfgekozen afmetingen enkel de grootste waarde over. Die grootste waarde geeft bovendien aan waar de filter het meest aanwezig was in de invoer. 

In het KIKS-netwerk wordt max pooling gebruikt in vensters van 2 pixels op 2 pixels. Hierdoor worden de feature maps vier keer kleiner door een max pooling-operatie.

De volgende afbeelding geeft een voorbeeld van zo'n max pooling-operatie.

<img src="images/maxpooling.jpg"/><br>
<center>Figuur 5: Max pooling.</center>

Het doel van max pooling-operaties is tweedelig. 
- Enerzijds wil men de grootte van de uitvoer van de convolutionele lagen verkleinen; waarom dit nodig is, wordt beschreven in de volgende paragraaf. 
- Anderzijds wil men meer informatie bekomen over een groter deel van de ingevoerde afbeelding. 

Stel je even voor dat de max pooling-operaties er niet zouden zijn. Na twee opeenvolgende convolutionele lagen zou een getal in de feature map informatie bevatten over een venster van slechts 5 op 5. Dit is vaak niet genoeg om belangrijke kenmerken van de afbeelding te herkennen. Zou je zelf iets in een afbeelding herkennen met slechts een 25-tal pixels?

Volgende afbeelding toont wat er gebeurt met twee convolutionele lagen (met steeds maar 1 filter) zonder max pooling-operaties. Het rode veld in de laatste feature map bevat enkel informatie uit het rode veld uit de ingevoerde afbeelding.

<img src="images/convnomaxpooling.jpg"/>
<center>Figuur 6: Convolutie zonder max pooling.</center>

### Opdracht
Wat is de grootte van het venster waarnaar een getal in de feature map verwijst, als er na beide convoluties telkens max pooling wordt uitgevoerd?

<div class="alert alert-block alert-warning"> 
Lees meer over max pooling in de notebook 'ReLU en max pooling' van het leerpad 'Basis Deep Learning'.
</div>

<div>
    <font color=#690027 markdown="1">
        <h3>2.4 Dense layers</h3> 
    </font>
</div>

De dense layers dienen om de afbeelding effectief te classificeren. 

Een dense layer bestaat uit neuronen waarbij elk neuron is verbonden met elk neuron van de volgende feedforward laag.
Aan elke verbinding tussen de neuronen van de opeenvolgende lagen is een bepaald getal toegekend, het **gewicht** van de verbinding.<br> Je kan het geheel vergelijken met een lineaire functie die de invoer van een laag omzet naar de uitvoer van die laag, waarbij de neuronen de variabelen zijn van de functie en de gewichten van de verbindingen de coëfficiënten, m.a.w. een lineaire combinatie van de neuronen. 

Een netwerk <em>leert</em> door deze gewichten aan te passen a.d.h.v. de trainingdata. Hoe meer neuronen, hoe meer informatie het netwerk kan opslaan. Te veel neuronen zijn echter ook niet altijd goed. Het netwerk kan dan bv. irrelevante informatie over de trainingdata beginnen op te slaan en te overfitten (zie verder).

De elementen van de feature map van de laatste convolutionele laag vormen de invoer van de eerste dense layer. 

<div class="alert alert-block alert-warning"> 
Meer uitleg over het feit dat te veel neuronen echter ook niet altijd goed zijn, vind je in de notebook 'Overfitting' in het leerpad 'Basis Deep Learning'.
</div>

In Figuur 7 wordt een feedforward netwerk voorgesteld als een graaf: de knopen (de cirkels) stellen de neuronen voor en de bogen de verbindingen tussen de neuronen. 

<img src="images/ffn.jpg"/><br>
<center>Figuur 7: Feedforward lagen met neuronen en verbindingen.</center>

Tot nu toe is de ingevoerde afbeelding door de convolutionele lagen en de max pooling-operaties omgezet in een 3D-tensor die informatie bevat over verschillende relevante patronen in die afbeelding. 

Voordat deze data als invoer van de dense layers kunnen dienen, moet de 3D-tensor worden omgezet in een rijmatrix. Dit gebeurt door een <b>flatten</b>-operatie. Figuur 8 toont hoe de flatten-operatie een feature map met dimensie 2x3x3 omzet in een rijmatrix om als invoer te dienen voor de dense layers.

<img src="images/flatten.jpg"/>
<center>Figuur 8: Flatten.</center>

Nu is ook meteen duidelijk waarom de uitvoer van de convolutionele lagen niet te groot mag zijn. Neem bijvoorbeeld een flatten-operatie op een feature map met dimensie 64x100x100, dit geeft 640 000 invoeren voor de eerste dense layer. Als de eerste dense layer zelf bv. 64 neuronen bevat, zijn er tussen deze twee lagen alleen al bijna 4,1 miljoen gewichten die het netwerk moet leren.

In de voorstelling van het diep neurale netwerk wordt een dense layer voorgesteld als een paars balkje. Het getal onder het balkje stelt het aantal neuronen in deze laag voor.

<div>
    <font color=#690027 markdown="1">
        <h3>2.5 Uitvoer</h3> 
    </font>
</div>

De laatste laag bestaat uit slechts één neuron; het is de uitvoer of de 'voorspelling' van het netwerk. Het netwerk geeft een getal tussen 0 en 1 terug, waarbij 0 staat voor 'Geen stoma' en 1 voor 'Stoma'. Hoe dichter de uitvoer bij 1 ligt, hoe zekerder het netwerk is dat er op de ingevoerde afbeelding een stoma te zien is. De persoon die het netwerk opbouwt, kiest een drempelwaarde. Eens deze drempelwaarde wordt overschreden, wordt een foto ingedeeld bij de klasse 'Stoma'.

<div>
    <font color=#690027 markdown="1">
        <h2>3. Kies je netwerkarchitectuur</h2> 
    </font>
</div>

Nu je de verschillende onderdelen van een diep neuraal netwerk begrijpt, kan je zelf een netwerk samenstellen om de stomataclassificatie uit te voeren. Voer onderstaande code-cel uit om enkele parameters van het netwerk te kiezen.

In [ ]:
diep_neuraal_netwerk.kies_netwerk_parameters()

Om het gekozen netwerk te visualiseren voer je volgende instructie uit. Ben je niet tevreden of wil je andere zaken uitproberen, verander dan gerust de parameters hierboven en voer de instructie opnieuw uit.

In [ ]:
diep_neuraal_netwerk.toon_netwerk()

<div>
    <font color=#690027 markdown="1">
        <h2>4. Het netwerk trainen</h2> 
    </font>
</div>

Eens de netwerkarchitectuur gekozen is, kan dit netwerk getraind worden. Maar voordat je daarmee kunt starten, moet je eerst nog enkele keuzes maken.

<div>
    <font color=#690027 markdown="1">
        <h3>4.1 Epochs</h3> 
    </font>
</div>

Je moet het aantal <b>epochs</b> kiezen: hoeveel keer de volledige trainingdata verwerkt worden. Vaak moet je wat experimenteren met het aantal epochs om een goed resultaat te bekomen. De netwerken in deze notebook zijn getraind met 50 epochs.

*Zoals eerder gezegd, worden de netwerken in deze notebook hier niet getraind. Ze zijn al op voorhand getraind en de resultaten zijn opgeslagen in een databank, waardoor je onmiddellijk de kenmerken van het gekozen netwerk kunt bekijken.* 

<div>
    <font color=#690027 markdown="1">
        <h3>4.2 Loss functie</h3> 
    </font>
</div>

Je moet ook een loss functie kiezen. De loss functie is een functie van de gewichten. De waarde van de loss functie beschrijft hoe goed een netwerk presteert. Hoe lager de waarde van de loss functie, hoe dichter de uitvoer van het netwerk bij de gewenste uitvoer ligt. Telkens als de gewichten worden aangepast tijdens de training, zal dus ook de waarde van de loss functie veranderen. Bij de training gaat men op zoek naar de gewichten met de kleinste loss.

Bij de netwerken in deze notebook is er gekozen voor de loss functie ***binary crossentropy***. Deze loss functie is geschikt voor classificatieproblemen met 2 klassen (vandaar de 'binary'). 

<div class="alert alert-block alert-warning"> 
Wil je meer weten over de werking van deze loss functie? Steek dan je licht op in de handleiding van 'KIKS'.
</div>

<div>
    <font color=#690027 markdown="1">
        <h3>4.3 Optimizer</h3> 
    </font>
</div>

Een andere belangrijke keuze bij het trainen van een diep neuraal netwerk is de optimizer. Deze bepaalt op welke manier het netwerk leert. De optimizer bepaalt hoe de gewichten aangepast moeten worden om dichter bij het minimum van de loss functie te komen. 

Bij de netwerken in deze notebook is er gekozen om de ***stochastic gradient descent (SGD)*** optimizer te gebruiken. Zoals de naam al doet vermoeden, maakt SGD gebruik van de techniek 'gradient descent': de afgeleide van de loss functie wordt berekend om er een minimum van te vinden. 'Stochastic' betekent dat de afgeleide van de loss niet wordt berekend met de volledige trainingset, maar met slechts een willekeurig gekozen deel ervan. Dit gaat een stuk sneller dan wanneer de afgeleide van de loss wordt berekend met de volledige trainingset.

<div class="alert alert-block alert-warning"> 
Verdiep je in de techniek van 'gradient descent' in de notebook 'Gradient Descent' van het leerpad 'Gevorderd Deep Learning'. Je ziet er ook het effect van de 'learning rate'.
</div>

<div>
    <font color=#690027 markdown="1">
        <h3>4.4 Learning rate</h3> 
    </font>
</div>

De learning rate bepaalt hoe sterk de gewichten telkens aangepast worden, m.a.w. hoe groot de stappen zijn om in het minimum te geraken. <br>
Een kleine learning rate zal ervoor zorgen dat het netwerk traag naar het minimum nadert, dus traag leert, en met een te grote learning rate zal het netwerk het minimum niet vinden. Volgende afbeelding toont hoe gradient descent werkt in 2 dimensies (dus met slechts één gewicht). Bij een diep neuraal netwerk zijn er enorm veel gewichten en dus ook enorm veel dimensies, wat veel moeilijker voor te stellen is door een figuur.

<img src="images/gradientdescent.jpg" width="500"/>
<center>Figuur 9: Gradient descent.</center>

Voer onderstaande code-cel uit om de learning rate te kiezen waarmee het netwerk wordt getraind.

In [ ]:
diep_neuraal_netwerk.kies_training_parameters()

<div>
    <font color=#690027 markdown="1">
        <h2>5. Prestatie van het netwerk</h2> 
    </font>
</div>

<div>
    <font color=#690027 markdown="1">
        <h3>5.1 Loss, accuracy en baseline</h3> 
    </font>
</div>

Om een netwerk te beoordelen kan je je dus baseren op de waarde van de loss functie, maar je kan ook kijken naar de <b>accuracy</b> of nauwkeurigheid van het netwerk, dit is het percentage van de data waarvoor het netwerk het label juist voorspelt. <br>
Beide waarden laat je tijdens het trainen, per epoch, berekenen voor zowel de trainingdata als de valideringsdata. <br>
Na het trainen laat je deze waarden nog één keer berekenen voor de testdata om het netwerk finaal te beoordelen.

Het kan ook gebeuren dat het netwerk niet leert. Om dit vast te stellen formuleer je een <b>baseline</b>. Wat dit is, is makkelijk uit te leggen aan de hand van het KIKS-voorbeeld. Onze dataset bevat 6 keer zoveel afbeeldingen zonder stoma als afbeeldingen met een stoma. In totaal is dus 6/7 (85,7 %) van de trainingset, validatieset en testset een afbeelding zonder stoma. Wanneer het model dus afbeeldingen als 'Geen stoma' aangeeft, zal er dus al een accuracy van 85,7 % zijn. Dit is de baseline die het model moet overtreffen voordat je kunt zeggen dat het model iets heeft geleerd.

Door volgende code-cel uit te voeren krijg je grafieken van de accuracy en de loss van het netwerk te zien, voor de training- en voor de valideringsset. De grafieken geven de accuracy en de loss weer per epoch.

In [ ]:
diep_neuraal_netwerk.toon_grafiek()

Het valt voor dat de training loss daalt, terwijl de validatie loss stijgt. Wanneer dit gebeurt, is het netwerk aan het <b>overfitten</b>. Het betekent dat het netwerk te veel details van de trainingdata uit het hoofd leert en daarom niet goed meer generaliseert op data die het nog nooit gezien heeft. <br>
Overfitting is een van de grootste problemen van een diep neuraal netwerk. Gelukkig bestaan er technieken om overfitting tegen te gaan. Een volgende notebook 'Overfitting' legt er een aantal uit.

Het tegenovergestelde van overfitten is <b>underfitten</b>: dit wil zeggen dat het netwerk niet genoeg geleerd heeft en daarom de relevante patronen in de data maar slecht kan herkennen. Vaak is dit het geval bij een te simpel netwerk of een netwerk dat te kort werd getraind.

<div class="alert alert-block alert-warning"> 
Bekijk technieken om overfitting tegen te gaan in de notebook 'Overfitting', ook in het leerpad 'Deep learning basis'.<br>
</div>

Figuur 10 toont hoe je underfitting en overfitting kan herkennen op een grafiek waarbij de waarden van de loss functie voor de verschillende epochs weergegeven zijn. Wanneer de training zou stoppen voor de meest linkse stippellijn, dan heb je een netwerk dat underfit. Wanneer de training zou stoppen na de meest rechtse stippellijn, heb je een netwerk dat overfit.

<img src="images/underfittingoverfitting.jpg"/>
<center>Figuur 10: Under- en overfitting.</center>

<div>
    <font color=#690027 markdown="1">
        <h3>5.2 Oefening</h3> 
    </font>
</div>

Zoek een model dat underfit, een model dat overfit en een model dat niet leert. Doe dit door in '3. Kies je netwerkarchitectuur' en '4.4 Learning rate' de parameters aan te passen en in '5.1. Loss, accuracy en baseline' de grafieken te bekijken. 

<div>
    <font color=#690027 markdown="1">
        <h3>5.3 Drempelwaarde</h3> 
    </font>
</div>

Als extra bekijk je nog een aantal voorspellingen van het netwerk. 

De voorspelling of uitvoer is een getal tussen 0 en 1 dat aangeeft hoe zeker het netwerk is dat er een stoma gevonden werd. De drempelwaarde bepaalt voor welke waarden van de uitvoer het netwerk de invoer als een stoma beschouwt. Als de  drempelwaarde bijvoorbeeld 0.5 is, zal een afbeelding met uitvoer groter dan 0.5 als "Stoma" beschouwd worden en alle afbeeldingen met uitvoer kleiner dan 0.5 als "Geen stoma". 

### Opdracht
- Voer onderstaande code-cel uit om de voorspellingen te zien. 
- Speel met de drempelwaarde (thr: weergegeven in procent).
- Kijk of het netwerk de afbeelding juist classificeert (een groene rand) of fout classificeert (een rode rand).

In [ ]:
diep_neuraal_netwerk.toon_voorspellingen()

<div>
    <font color=#690027 markdown="1">
        <h3>5.4 Vals positief en vals negatief</h3> 
    </font>
</div>

Er zijn ook afbeeldingen waar het model het moeilijk mee heeft. Wanneer het netwerk oordeelt dat op een foto met een stoma geen stoma staat, spreekt men van een **vals negatief**. Omgekeerd, wanneer het netwerk oordeelt dat er op een foto zonder stoma wel een stoma staat, spreekt men van een **vals positief**. 

Voer onderstaande code-cel uit om enkele afbeeldingen te zien waarmee veel modellen het moeilijk hebben.

In [ ]:
diep_neuraal_netwerk.toon_slechte_voorspellingen()

<div>
    <font color=#690027 markdown="1">
        <h3>5.5 Prestatie op de testset</h3> 
    </font>
</div>

Ben je tevreden over je netwerk, dan ga je het nog een laatste keer beoordelen aan de hand van de testset. Voeg volgende code-cel uit om de uiteindelijke prestatie van je netwerk te zien.

In [ ]:
diep_neuraal_netwerk.toon_test_resultaten()

<div class="alert alert-block alert-warning"> 
In de notebook 'Van blad naar label' van het leerpad 'Gevorderd Deep Learning' experimenteer je met de parameters van het KIKS-neuraal netwerk. Je traint m.a.w. je eigen AI-systeem dat huidmondjes herkent en telt. Je streeft naar een zo accuraat en efficiënt mogelijk netwerk. 
</div>

<div>
    <h2>Met steun van</h2> 
</div>

<img src="images/kikssteun.png" alt="Banner" width="1100"/>

<img src="images/cclic.png" alt="Banner" align="left" width="100"/><br><br>
Notebook KIKS, zie <a href="http://www.aiopschool.be">AI Op School</a>, van F. wyffels, A. Meheus, T. Neutens & N. Gesquière is in licentie gegeven volgens een <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Naamsvermelding-NietCommercieel-GelijkDelen 4.0 Internationaal-licentie</a>. 